- BOW
- TFIDF
- Word2Vec
    - Glove
- Doc2Vec
- Transformers

In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
print(len(stop))

179


[nltk_data] Downloading package stopwords to C:\Users\ALIENWARE
[nltk_data]     M15\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity

In [3]:
df = pd.read_csv('titles.csv')
print(df.shape)
df.head()

(119453, 1)


,title
0,Solving QBF Instances with Nested SAT Solvers.
1,Multiscale Manifold Learning.
2,Online Transfer Learning for Differential Diag...
3,Logic Programming in Assumption-Based Argument...
4,Role-Aware Conformity Modeling and Analysis in...


In [ ]:
# If the csv isn't working, run this cell
import pickle

df2 = pd.DataFrame()
with open('original_titles.pkl', 'rb') as f:
    df2['title'] = pickle.load(f)

df2.head()

In [19]:
sample = df.sample(50)
sample = sample.reset_index(drop=True)
sample.head()

,title
0,Non-Linear Regression for Bag-of-Words Data vi...
1,Time-Aware Rank Aggregation for Microblog Search.
2,Weakly-supervised relation classification for ...
3,Tap control for headphones without sensors.
4,Processing High-Volume Stream Queries on a Sup...


In [20]:
# Cleaning the data
sample['text'] = sample['title'].str.replace('[^a-zA-Z]', ' ') # Remove punctuation
sample['text'] = sample['text'].str.lower() # Lowercase
sample['text'] = sample['text'].apply(lambda x: " ".join([word for word in x.split() if word not in (stop)])) # Remove stopwords
sample.head()

,title,text
0,Non-Linear Regression for Bag-of-Words Data vi...,non linear regression bag words data via gauss...
1,Time-Aware Rank Aggregation for Microblog Search.,time aware rank aggregation microblog search
2,Weakly-supervised relation classification for ...,weakly supervised relation classification info...
3,Tap control for headphones without sensors.,tap control headphones without sensors
4,Processing High-Volume Stream Queries on a Sup...,processing high volume stream queries supercom...


In [6]:
model = SentenceTransformer('sentence-transformers/multi-qa-MiniLM-L6-cos-v1')

Downloading: 100%|██████████| 190/190 [00:00<00:00, 38.0kB/s]
Downloading: 100%|██████████| 11.5k/11.5k [00:00<00:00, 1.65MB/s]
Downloading: 100%|██████████| 116/116 [00:00<00:00, 116kB/s]
Downloading: 100%|██████████| 25.5k/25.5k [00:00<00:00, 82.2kB/s]
Downloading: 100%|██████████| 90.9M/90.9M [00:25<00:00, 3.53MB/s]
Downloading: 100%|██████████| 53.0/53.0 [00:00<00:00, 13.1kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 18.7kB/s]
Downloading: 100%|██████████| 466k/466k [00:01<00:00, 295kB/s]  
Downloading: 100%|██████████| 383/383 [00:00<00:00, 95.2kB/s]
Downloading: 100%|██████████| 13.8k/13.8k [00:00<00:00, 44.5kB/s]
Downloading: 100%|██████████| 232k/232k [00:01<00:00, 187kB/s]  
Downloading: 100%|██████████| 349/349 [00:00<00:00, 172kB/s]


In [21]:
features = model.encode(sample['text'])
sample['features'] = list(features)
sample.head()

,title,text,features
0,Non-Linear Regression for Bag-of-Words Data vi...,non linear regression bag words data via gauss...,"[0.005292843, -0.055277072, -0.032679874, -0.0..."
1,Time-Aware Rank Aggregation for Microblog Search.,time aware rank aggregation microblog search,"[0.013465883, -0.055515103, -0.035684034, -0.0..."
2,Weakly-supervised relation classification for ...,weakly supervised relation classification info...,"[-0.0033559168, 0.05058709, -0.09418492, -0.02..."
3,Tap control for headphones without sensors.,tap control headphones without sensors,"[0.054999277, 0.018125866, 0.0070430795, -0.00..."
4,Processing High-Volume Stream Queries on a Sup...,processing high volume stream queries supercom...,"[0.03355885, -0.045568187, -0.064688735, -0.05..."


In [23]:
def search(query_text):
    query_feature = model.encode([query_text])
    sample['similarity'] = sample['features'].apply(lambda x: cosine_similarity([x], query_feature)[0][0]) 
    sorted = sample.sort_values(by='similarity', ascending=False)
    return sorted.head(10)

In [ ]:
search('deep learning')

In [ ]:
search('Knowledge Bases for Web Content Analytics.')

In [ ]:
search(str(input('Enter a query: ')))

In [33]:
def results(Google_Search_Engine_Clone):
    query_feature = model.encode([Google_Search_Engine_Clone])
    sample['similarity'] = sample['features'].apply(lambda x: cosine_similarity([x], query_feature)[0][0]) 
    sorted = sample.sort_values(by='similarity', ascending=False)
    result = sorted['title'].head(5)
    return list(result)

In [30]:
results("Maths")

['Towards a Mathematical Theory of Primal Sketch and Sketchability.',
 'Open Problem: Shifting Experts on Easy Data.',
 'A New Approach to Online Scheduling: Approximating the Optimal Competitive Ratio.',
 'Understanding finger input above desktop devices.',
 'Private approximation of search problems.']

In [34]:
import gradio as gr

u_interface = gr.Interface(fn=results, inputs="text", outputs="text")
u_interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://0c3eb6bb1735644c.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x1c69eb33190>,
 'http://127.0.0.1:7862/',
 'https://0c3eb6bb1735644c.gradio.app')